## Data process

Code to process AIDA-CONLL data.

Output json file:

{'doc_id': document id,

 'text': document text,
 
 'mentions': ['EU', 'German', ...],
 
 'mention_ranges': [[1,2],[3,4],...],
 
 'wiki_ids': [0000, 0001, ...]}

Reference: https://github.com/facebookresearch/BLINK/blob/master/scripts/create_BLINK_benchmark_data.py

In [71]:
import json
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchtext
from bs4 import BeautifulSoup
from tqdm import tqdm

%matplotlib inline

In [106]:
# BEGIN_ENT_TOKEN = "[START_ENT]"
# END_ENT_TOKEN = "[END_ENT]"

#url2id_cache = {}

# with open('url2id.json', 'w') as fout:
#     json.dump(url2id_cache, fout)

with open('url2id.json') as fin:
    url2id_cache = json.load(fin)

In [5]:
def _read_url(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        soup = BeautifulSoup(html, features="html.parser")
        title = soup.title.string.replace(" - Wikipedia", "").strip()
    return title

In [4]:
def _get_pageid_from_api(title, client=None):
    pageid = None

    title_html = title.strip().replace(" ", "%20")
    url = "https://en.wikipedia.org/w/api.php?action=query&titles={}&format=json".format(
        title_html
    )

    try:
        # Package the request, send the request and catch the response: r
        r = requests.get(url)

        # Decode the JSON data into a dictionary: json_data
        json_data = r.json()

        if len(json_data["query"]["pages"]) > 1:
            print("WARNING: more than one result returned from wikipedia api")

        for _, v in json_data["query"]["pages"].items():
            pageid = v["pageid"]
    except:
        pass

    return pageid

## Data with first 128 tokens of wikipedia content

In [78]:
id2desc_cache = {}

In [8]:
# read in data file without description
fpath = 'AIDA-YAGO2'

fname = os.path.join(fpath, 'train.json')
with open(fname) as fin:
    train = json.load(fin)

fname = os.path.join(fpath, 'dev.json')
with open(fname) as fin:
    dev = json.load(fin)

fname = os.path.join(fpath, 'test.json')
with open(fname) as fin:
    test = json.load(fin)

In [9]:
len(train), len(dev), len(test)

(946, 216, 231)

In [19]:
datasets = [train, dev, test]
id2title = {}

for dataset in datasets:
    for doc in dataset:
        ids = doc['wiki_ids']
        titles = doc['wiki_titles']
        assert len(ids)==len(titles)
        for i in range(len(ids)):
            id2title[ids[i]] = titles[i]

In [21]:
#len(wiki_ids)
len(id2title)

5598

In [84]:
def get_content_from_api(str_id, tokenizer=tokenizer, max_len=128, client=None):

    url2 = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext&redirects=1&pageids={}".format(
        str_id
    )
    try:
        r = requests.get(url2)
        json_data = r.json()

        if len(json_data["query"]["pages"]) > 1:
            print("WARNING: more than one result returned from wikipedia api")

        page = json_data['query']['pages'][str_id]
        title = page['title']

        if title!=id2title[str_id]:
            print(str_id, '  ', title, '  ', id2title[str_id])

        content = page['extract']
        tokens = tokenizer(content)
        return ' '.join(tokens[:128])
    except:
        print(str_id)
        return None

In [81]:
tokenizer = torchtext.data.get_tokenizer("basic_english")

In [85]:
for str_id in tqdm(id2title):
    if str_id in id2desc_cache:
        continue
    token_content = get_content_from_api(str_id, tokenizer)
    id2desc_cache[str_id] = token_content

  1%|          | 34/5598 [00:00<00:28, 192.38it/s]

11095


  1%|          | 59/5598 [00:52<2:23:15,  1.55s/it]

3550889    Channel 2 (Israeli TV channel)    Channel 2 (Israel)


  1%|          | 66/5598 [01:02<1:32:56,  1.01s/it]

...

100%|█████████▉| 5583/5598 [2:06:03<00:28,  1.91s/it]

5834903    Hapoel Be'er Sheva F.C.    Hapoel Be'er Sheva A.F.C.


100%|█████████▉| 5587/5598 [2:06:09<00:16,  1.47s/it]

243389    Charlotte Hornets    Charlotte Bobcats


100%|██████████| 5598/5598 [2:06:20<00:00,  1.35s/it]


In [99]:
# 'China', 'Taiwan', 'Frankfurt'

In [86]:
with open('id2desc.json', 'w') as fout:
    json.dump(id2desc_cache, fout)

In [92]:
sum_none = 0
for key in id2desc_cache:
    if id2desc_cache[key] is None:
        sum_none += 1

print(sum_none)

83


In [96]:
def process_data_wiki_content(filename, id2desc, debug=False):
    """
     Input: AIDA-YAGO2-dataset.tsv, columns: token, B_I tag, mention, YAGO2 entity, wiki url, wiki id 
     Output: processed dictionary of document, context of labels from first 128 tokens of wikipedia content
    """
    processed = {}

    open_entity = False

    with open(filename) as fin:
        lines = fin.readlines()
        if debug:
            lines = lines[:100]

        for line in tqdm(lines):
            if '-DOCSTART-' in line:
                # check end of entity (last sentence of doc)
                if open_entity:
                    processed[doc_id]['end_idxs'][-1] = 1
                    open_entity = False
                
                # new document
                doc_id = line.split("(")[-1][:-2]
                processed[doc_id] = {
                    'doc_id': doc_id,
                    'text': [],
                    'mentions': [],
                    'start_idxs': [],
                    'end_idxs': [],
                    'wiki_ids': [],
                    'wiki_titles': [],
                    'wiki_contexts': []
                }
            else:
                split = line.split('\t')
                token = split[0].strip()

                if len(token)==0: # space are eliminated
                    continue

                processed[doc_id]['text'] += [token]

                if len(split)>=5:
                    B_I = split[1]
                    mention = split[2]
                    wiki_url = split[4]
                    wiki_id = split[5].split('\n')[0]

                    if B_I == 'I':
                        processed[doc_id]['start_idxs'] += [0]
                        processed[doc_id]['end_idxs'] += [0]
                    elif B_I == 'B':
                        # check for consecutive mentions, such as B I B
                        if open_entity:
                            processed[doc_id]['end_idxs'][-1] = 1
                            open_entity = False
                        
                        # check for entity returned None for wikipedia content
                        # ignore this mention
                        if id2desc[wiki_id] is None:
                            processed[doc_id]['start_idxs'] += [0]
                            processed[doc_id]['end_idxs'] += [0]
                        else:
                            wiki_title = wiki_url.split('/')[-1].replace('_', ' ')
                            if wiki_id == '000':
                                if wiki_url in url2id_cache:
                                    wiki_id = urlid_cache[wiki_url]
                                else:
                                    wiki_id = _get_pageid_from_api(wiki_title)
                                    urlid_cache[wiki_url] = wiki_id
                            wiki_context = id2desc[wiki_id]
                            q = processed[doc_id]
                            q['mentions'] += [mention]
                            q['start_idxs'] += [1]
                            q['end_idxs'] += [0]
                            q['wiki_ids'] += [wiki_id]
                            q['wiki_titles'] += [wiki_title]
                            q['wiki_contexts'] += [wiki_context]
                            processed[doc_id] = q
                            open_entity = True
                    else:
                        print('Invalid B_I {}'.format(B_I))
                        sys.exit(-1)
                else:
                    if open_entity:
                        processed[doc_id]['end_idxs'][-1] = 1
                        open_entity = False
                    processed[doc_id]['start_idxs'] += [0]
                    processed[doc_id]['end_idxs'] += [0]
#     # final sentence
#     if open_entity:
#         processed[doc_id]['end_idxs'][-1] = 1
#         open_entity = False

    return processed
            

In [97]:
filename = 'AIDA-YAGO2-dataset.tsv'
processed_aida = process_data_wiki_content(filename, id2desc_cache)

100%|██████████| 323395/323395 [00:05<00:00, 56623.98it/s] 


In [98]:
# sanity check of processed data
for key, d in tqdm(processed_aida.items()):
    try:
        assert len(d['text'])==len(d['start_idxs'])
        assert len(d['start_idxs'])==len(d['end_idxs'])
        assert len(d['mentions'])==sum(d['start_idxs'])
        assert len(d['mentions'])==sum(d['end_idxs'])
        assert len(d['wiki_ids'])==len(d['mentions'])
        assert len(d['wiki_titles'])==len(d['wiki_ids'])
        assert len(d['wiki_ids'])==len(d['wiki_contexts'])
    except:
        print(key)

100%|██████████| 1393/1393 [00:00<00:00, 126063.51it/s]


In [100]:
train = []
testa = []
testb = []

for key, q in tqdm(processed_aida.items()):
    if 'testa' in key:
        testa.append(q)
    elif 'testb' in key:
        testb.append(q)
    else:
        train.append(q)

print("train: {}".format(len(train)))
print("testa: {}".format(len(testa)))
print("testb: {}".format(len(testb)))

100%|██████████| 1393/1393 [00:00<00:00, 312826.76it/s]

train: 946
testa: 216
testb: 231


In [101]:
def mean(l):
    return sum(l)/len(l)

avg_token_train = mean([len(item['text']) for item in train])
avg_token_testa = mean([len(item['text']) for item in testa])
avg_token_testb = mean([len(item['text']) for item in testb])

print(avg_token_train, avg_token_testa, avg_token_testb)

215.24418604651163 237.78703703703704 201.017316017316


In [102]:
def add_mention_ranges(items):
    for item in tqdm(items):
        start_position = [i for i in range(len(item['start_idxs'])) if item['start_idxs'][i]==1]
        end_position = [i for i in range(len(item['end_idxs'])) if item['end_idxs'][i]==1]
        assert len(start_position)==len(end_position)
        span_position = [[start_position[i], end_position[i]] for i in range(len(start_position))]
        item['start_position'] = start_position
        item['end_position'] = end_position
        item['span_position'] = span_position
    return items

In [103]:
train = add_mention_ranges(train)

100%|██████████| 946/946 [00:00<00:00, 9556.71it/s]


In [104]:
dev = add_mention_ranges(testa)
test = add_mention_ranges(testb)

100%|██████████| 231/231 [00:00<00:00, 21717.04it/s]


In [105]:
fpath = 'AIDA-YAGO2-wiki_content'

fname = os.path.join(fpath, 'train.json')
with open(fname, 'w') as fout:
    json.dump(train, fout)

fname = os.path.join(fpath, 'dev.json')
with open(fname, 'w') as fout:
    json.dump(dev, fout)

fname = os.path.join(fpath, 'test.json')
with open(fname, 'w') as fout:
    json.dump(test, fout)

## Data with en_description from Kensho wiki data

In [7]:
# import wiki knowledge (en_description)
all_df = pd.read_csv('wiki/all_df.csv')
all_df

,Unnamed: 0,page_id,item_id,title,views,en_label,en_description
0,0,12,6199,Anarchism,31335,anarchism,political ideology that holds all government –...
1,1,25,38404,Autism,49693,autism,neurodevelopmental condition
2,2,39,101038,Albedo,14573,albedo,ratio of reflected radiation to incident radia...
3,3,290,9659,A,25859,A,letter of the Latin alphabet
4,4,303,173,Alabama,52765,Alabama,state of the United States of America
...,...,...,...,...,...,...,...
5362169,5362169,62470350,76894635,Daming Zhu,16,Daming Zhu,NaN
5362170,5362170,62470423,76894633,Tony Dews,7,Tony Dews,American football coach
5362171,5362171,62470432,76896959,Samsung PL20,9,Samsung PL20,NaN
5362172,5362172,62470465,6034153,Nils-Fredrik Palmstierna,8,Nils-Fredrik Palmstierna,NaN


In [11]:
page_ids = all_df.page_id.values
page_ids

array([      12,       25,       39, ..., 62470432, 62470465, 62473330])

In [18]:
all_df_no_na = all_df.dropna(subset=['en_description'])
all_df_no_na

,Unnamed: 0,page_id,item_id,title,views,en_label,en_description
0,0,12,6199,Anarchism,31335,anarchism,political ideology that holds all government –...
1,1,25,38404,Autism,49693,autism,neurodevelopmental condition
2,2,39,101038,Albedo,14573,albedo,ratio of reflected radiation to incident radia...
3,3,290,9659,A,25859,A,letter of the Latin alphabet
4,4,303,173,Alabama,52765,Alabama,state of the United States of America
...,...,...,...,...,...,...,...
5362162,5362162,62470001,54862857,James Daniel Collins,6,James Daniel Collins,1917-1985
5362164,5362164,62470047,76852005,Tie Vapauteen (album),12,Tie vapauteen,album by Paleface & Laulava unioni
5362165,5362165,62470119,76891681,Sumner Heights and Hazelwood Valley Railroad,16,Sumner Heights and Hazelwood Valley Railroad,minimum gauge railway near Boston
5362170,5362170,62470423,76894633,Tony Dews,7,Tony Dews,American football coach


In [16]:
all_df.loc[all_df.page_id==25]['en_description'].values[0]

'neurodevelopmental condition'

In [17]:
sum(pd.isna(all_df['en_description']))

1491956

## todo:
### for single word mention, would start idx and end idx the same?

#### currently: the same

In [47]:
def process_data_en_description(filename, all_df, debug=False):
    """
     Input: AIDA-YAGO2-dataset.tsv, columns: token, B_I tag, mention, YAGO2 entity, wiki url, wiki id 
     Output: processed dictionary of document, context of labels from en_description
    """
    processed = {}

    open_entity = False
    
    # all_df: no N/A value in en_description
    page_ids = all_df.page_id.values

    with open(filename) as fin:
        lines = fin.readlines()
        if debug:
            lines = lines[:100]

        for line in tqdm(lines):
            if '-DOCSTART-' in line:
                # check end of entity (last sentence of doc)
                if open_entity:
                    processed[doc_id]['end_idxs'][-1] = 1
                    open_entity = False
                
                # new document
                doc_id = line.split("(")[-1][:-2]
                processed[doc_id] = {
                    'doc_id': doc_id,
                    'text': [],
                    'mentions': [],
                    'start_idxs': [],
                    'end_idxs': [],
                    'wiki_ids': [],
                    'wiki_titles': [],
                    'wiki_contexts': []
                }
            else:
                split = line.split('\t')
                token = split[0].strip()

                if len(token)==0: # space are eliminated
                    continue

                processed[doc_id]['text'] += [token]

                if len(split)>=5:
                    B_I = split[1]
                    mention = split[2]
                    wiki_url = split[4]
                    wiki_id = int(split[5].split('\n')[0])

                    if B_I == 'I':
                        processed[doc_id]['start_idxs'] += [0]
                        processed[doc_id]['end_idxs'] += [0]
                    elif B_I == 'B':
                        # check for consecutive mentions, such as B I B
                        if open_entity:
                            processed[doc_id]['end_idxs'][-1] = 1
                            open_entity = False
                        
                        # check for entity not in Kensho wiki knowledge base
                        # ignore this mention
                        if wiki_id not in page_ids:
                            processed[doc_id]['start_idxs'] += [0]
                            processed[doc_id]['end_idxs'] += [0]
                        else:
                            wiki_title = wiki_url.split('/')[-1].replace('_', ' ')
                            if str(wiki_id) == '000':
                                if wiki_url in url2id_cache:
                                    wiki_id = urlid_cache[wiki_url]
                                else:
                                    wiki_id = _get_pageid_from_api(wiki_title)
                                    urlid_cache[wiki_url] = wiki_id
                            wiki_context = all_df.loc[all_df.page_id==wiki_id]['en_description'].values[0]
                            q = processed[doc_id]
                            q['mentions'] += [mention]
                            q['start_idxs'] += [1]
                            q['end_idxs'] += [0]
                            q['wiki_ids'] += [wiki_id]
                            q['wiki_titles'] += [wiki_title]
                            q['wiki_contexts'] += [wiki_context]
                            processed[doc_id] = q
                            open_entity = True
                    else:
                        print('Invalid B_I {}'.format(B_I))
                        sys.exit(-1)
                else:
                    if open_entity:
                        processed[doc_id]['end_idxs'][-1] = 1
                        open_entity = False
                    processed[doc_id]['start_idxs'] += [0]
                    processed[doc_id]['end_idxs'] += [0]
#     # final sentence
#     if open_entity:
#         processed[doc_id]['end_idxs'][-1] = 1
#         open_entity = False

    return processed
            

In [48]:
filename = 'AIDA-YAGO2-dataset.tsv'
#processed_aida = process_data(filename, debug=True)
#processed_aida = process_data(filename)
processed_aida = process_data_en_description(filename, all_df_no_na)

100%|██████████| 323395/323395 [04:05<00:00, 1316.89it/s]


In [49]:
# sanity check of processed data
for key, d in tqdm(processed_aida.items()):
    try:
        assert len(d['text'])==len(d['start_idxs'])
        assert len(d['start_idxs'])==len(d['end_idxs'])
        assert len(d['mentions'])==sum(d['start_idxs'])
        assert len(d['mentions'])==sum(d['end_idxs'])
        assert len(d['wiki_ids'])==len(d['mentions'])
        assert len(d['wiki_titles'])==len(d['wiki_ids'])
        assert len(d['wiki_ids'])==len(d['wiki_contexts'])
    except:
        print(key)

100%|██████████| 1393/1393 [00:00<00:00, 56534.45it/s]


In [50]:
train = []
testa = []
testb = []

for key, q in tqdm(processed_aida.items()):
    if 'testa' in key:
        testa.append(q)
    elif 'testb' in key:
        testb.append(q)
    else:
        train.append(q)

print("train: {}".format(len(train)))
print("testa: {}".format(len(testa)))
print("testb: {}".format(len(testb)))

100%|██████████| 1393/1393 [00:00<00:00, 269123.24it/s]

train: 946
testa: 216
testb: 231


In [51]:
def mean(l):
    return sum(l)/len(l)

In [52]:
avg_token_train = mean([len(item['text']) for item in train])
avg_token_testa = mean([len(item['text']) for item in testa])
avg_token_testb = mean([len(item['text']) for item in testb])

In [53]:
print(avg_token_train, avg_token_testa, avg_token_testb)

215.24418604651163 237.78703703703704 201.017316017316


In [54]:
def add_mention_ranges(items):
    for item in tqdm(items):
        start_position = [i for i in range(len(item['start_idxs'])) if item['start_idxs'][i]==1]
        end_position = [i for i in range(len(item['end_idxs'])) if item['end_idxs'][i]==1]
        assert len(start_position)==len(end_position)
        span_position = [[start_position[i], end_position[i]] for i in range(len(start_position))]
        item['start_position'] = start_position
        item['end_position'] = end_position
        item['span_position'] = span_position
    return items

In [55]:
train = add_mention_ranges(train)

100%|██████████| 946/946 [00:00<00:00, 17302.28it/s]


In [56]:
dev = add_mention_ranges(testa)

100%|██████████| 216/216 [00:00<00:00, 15258.69it/s]


In [57]:
test = add_mention_ranges(testb)

100%|██████████| 231/231 [00:00<00:00, 11137.88it/s]


In [58]:
fpath = 'AIDA-YAGO2-en_desc'

fname = os.path.join(fpath, 'train.json')
with open(fname, 'w') as fout:
    json.dump(train, fout)

fname = os.path.join(fpath, 'dev.json')
with open(fname, 'w') as fout:
    json.dump(dev, fout)

fname = os.path.join(fpath, 'test.json')
with open(fname, 'w') as fout:
    json.dump(test, fout)

### Check mention counts before and after add en_description/first 128 tokens

In [107]:
with open('AIDA-YAGO2/train.json') as f:
    train_orig = json.load(f)

with open('AIDA-YAGO2-en_desc/train.json') as f:
    train_en = json.load(f)

with open('AIDA-YAGO2-wiki_content/train.json') as f:
    train_wiki = json.load(f)

In [108]:
train_orig_mentions = [len(i['mentions']) for i in train_orig]
train_en_mentions = [len(i['mentions']) for i in train_en]
train_wiki_mentions = [len(i['mentions']) for i in train_wiki]

In [109]:
comp_df = pd.DataFrame({'orig':train_orig_mentions, 'en_desc':train_en_mentions, 'wiki':train_wiki_mentions})
comp_df

,orig,en_desc,wiki
0,30,30,30
1,11,11,11
2,24,24,24
3,7,7,7
4,12,11,11
...,...,...,...
941,27,27,27
942,27,26,27
943,33,33,33
944,47,45,47


In [110]:
comp_df.describe()

,orig,en_desc,wiki
count,946.000000,946.000000,946.000000
mean,19.599366,19.079281,19.355180
std,21.674191,21.528843,21.558648
min,0.000000,0.000000,0.000000
25%,7.000000,7.000000,7.000000
50%,13.000000,13.000000,13.000000
75%,26.000000,25.000000,25.000000
max,288.000000,286.000000,286.000000


In [111]:
comp_df['en_diff'] = comp_df['orig']-comp_df['en_desc']
comp_df['wiki_diff'] = comp_df['orig']-comp_df['wiki']
comp_df

,orig,en_desc,wiki,en_diff,wiki_diff
0,30,30,30,0,0
1,11,11,11,0,0
2,24,24,24,0,0
3,7,7,7,0,0
4,12,11,11,1,1
...,...,...,...,...,...
941,27,27,27,0,0
942,27,26,27,1,0
943,33,33,33,0,0
944,47,45,47,2,0


In [112]:
comp_df['en_diff'].value_counts()

0     711
1     120
2      63
3      19
4      16
8       4
5       4
6       3
9       2
7       2
12      1
11      1
Name: en_diff, dtype: int64

In [114]:
comp_df['wiki_diff'].value_counts()

0     838
1      57
2      29
4       7
3       5
9       3
7       2
6       2
5       2
10      1
Name: wiki_diff, dtype: int64

### Manual check

In [115]:
list(train[0].keys())

['doc_id',
 'text',
 'mentions',
 'start_idxs',
 'end_idxs',
 'wiki_ids',
 'wiki_titles',
 'wiki_contexts',
 'start_position',
 'end_position',
 'span_position']

In [116]:
text = train[0]['text']
spans = train[0]['span_position']
mention = [text[span[0]:span[1]+1] for span in spans]
mention = [' '.join(m) for m in mention]

In [117]:
train[0]['mentions']

['German',
 'British',
 'BRUSSELS',
 'European Commission',
 'German',
 'British',
 'Germany',
 'European Union',
 'Britain',
 'Commission',
 'European Union',
 'Franz Fischler',
 'Britain',
 'France',
 'BSE',
 'Spanish',
 'Loyola de Palacio',
 'France',
 'Britain',
 'BSE',
 'British',
 'German',
 'British',
 'Europe',
 'Germany',
 'Bonn',
 'British',
 'Germany',
 'Britain',
 'British']

In [118]:
mention

['German',
 'British',
 'BRUSSELS',
 'European Commission',
 'German',
 'British',
 'Germany',
 'European Union',
 'Britain',
 'Commission',
 'European Union',
 'Franz Fischler',
 'Britain',
 'France',
 'BSE',
 'Spanish',
 'Loyola de Palacio',
 'France',
 'Britain',
 'BSE',
 'British',
 'German',
 'British',
 'Europe',
 'Germany',
 'Bonn',
 'British',
 'Germany',
 'Britain',
 'British']

In [119]:
train[0]['mentions'] == mention

True

In [120]:
train[0]['span_position']

[[2, 2],
 [6, 6],
 [11, 11],
 [14, 15],
 [22, 22],
 [28, 28],
 [43, 43],
 [48, 49],
 [66, 66],
 [94, 94],
 [129, 130],
 [142, 143],
 [179, 179],
 [181, 181],
 [193, 193],
 [237, 237],
 [240, 242],
 [266, 266],
 [268, 268],
 [314, 314],
 [330, 330],
 [350, 350],
 [357, 357],
 [363, 363],
 [382, 382],
 [404, 404],
 [420, 420],
 [438, 438],
 [443, 443],
 [459, 459]]

In [123]:
train[0]['wiki_contexts']

['germany ( german deutschland , german pronunciation [ˈdɔʏtʃlant] ) , officially the federal republic of germany ( german bundesrepublik deutschland , listen ) , is a country at the intersection of central and western europe . it is situated between the baltic and north seas to the north , and the alps to the south . it borders denmark to the north , poland and the czech republic to the east , austria and switzerland to the south , and france , luxembourg , belgium , and the netherlands to the west , and covers an area of 357 , 022 square kilometres ( 137 , 847 sq mi ) . various germanic tribes have inhabited the northern parts of modern germany since classical antiquity . a region',
 'the united kingdom of great britain and northern ireland , commonly known as the united kingdom ( uk or u . k . ) , or britain , is a sovereign country in north-western europe , off the north-\xadwestern coast of the european mainland . the united kingdom includes the island of great britain , the north

# Draft

In [87]:
# store on file
def store_data(processed_data, OUT_FILENAME):
    with open(OUT_FILENAME, "w+") as fout:
        for q in processed_data:
            json.dump(q, fout)
            fout.write("\n")

In [89]:
out_train_aida_filename = "AIDA-YAGO2_train.jsonl"
store_data(train, out_train_aida_filename)
out_testa_aida_filename = "AIDA-YAGO2_testa.jsonl"
store_data(testa, out_testa_aida_filename)
out_testb_aida_filename = "AIDA-YAGO2_testb.jsonl"
store_data(testb, out_testb_aida_filename)

In [85]:
# check = processed_aida['51 BASEBALL']
# check

In [52]:
sum(check['start_idxs'])

57

In [53]:
sum(check['end_idxs'])

56

In [54]:
len(check['text'])

266

In [55]:
len(check['end_idxs'])

266

In [56]:
import numpy as np

In [57]:
start = np.array(check['start_idxs'])
end = np.array(check['end_idxs'])

In [79]:
mention_ranges = []

for i in range(len(start)):
    if start[i]==1:
        mention_ranges.append([i])
        for j in range(i, len(end)):
            if end[j]==1:
                mention_ranges[-1].append(j+1)
                break

In [80]:
mention_ranges

[[2, 4],
 [10, 12],
 [13, 16],
 [36, 40],
 [44, 46],
 [50, 51],
 [55, 56],
 [60, 61],
 [65, 66],
 [71, 73],
 [73, 74],
 [78, 79],
 [84, 85],
 [90, 91],
 [96, 98],
 [103, 105],
 [105, 106],
 [110, 111],
 [115, 116],
 [120, 121],
 [131, 132],
 [133, 134],
 [134, 135],
 [136, 137],
 [137, 138],
 [139, 141],
 [141, 142],
 [143, 144],
 [144, 145],
 [146, 148],
 [148, 149],
 [150, 151],
 [151, 155],
 [159, 160],
 [164, 165],
 [169, 171],
 [176, 177],
 [181, 182],
 [186, 188],
 [188, 189],
 [199, 200],
 [204, 205],
 [210, 211],
 [216, 218],
 [218, 220],
 [224, 226],
 [231, 232],
 [236, 238],
 [251, 252],
 [252, 253],
 [254, 255],
 [255, 256],
 [257, 258],
 [258, 259],
 [260, 262],
 [262, 263],
 [264]]

In [81]:
for x in mention_ranges:
    try:
        print(check['text'][x[0]:x[1]])
    except:
        print(check['text'][x[0]])

['MAJOR', 'LEAGUE']
['NEW', 'YORK']
['Major', 'League', 'Baseball']
['AMERICAN', 'LEAGUE', 'EASTERN', 'DIVISION']
['NEW', 'YORK']
['BALTIMORE']
['BOSTON']
['TORONTO']
['DETROIT']
['CENTRAL', 'DIVISION']
['CLEVELAND']
['CHICAGO']
['MINNESOTA']
['MILWAUKEE']
['KANSAS', 'CITY']
['WESTERN', 'DIVISION']
['TEXAS']
['SEATTLE']
['OAKLAND']
['CALIFORNIA']
['OAKLAND']
['BOSTON']
['SEATTLE']
['BALTIMORE']
['CALIFORNIA']
['NEW', 'YORK']
['TORONTO']
['CHICAGO']
['DETROIT']
['KANSAS', 'CITY']
['TEXAS']
['MINNESOTA']
['NATIONAL', 'LEAGUE', 'EASTERN', 'DIVISION']
['ATLANTA']
['MONTREAL']
['NEW', 'YORK']
['FLORIDA']
['PHILADELPHIA']
['CENTRAL', 'DIVISION']
['HOUSTON']
['CHICAGO']
['CINCINNATI']
['PITTSBURGH']
['WESTERN', 'DIVISION']
['SAN', 'DIEGO']
['LOS', 'ANGELES']
['COLORADO']
['SAN', 'FRANCISCO']
['COLORADO']
['CINCINNATI']
['ATLANTA']
['PITTSBURGH']
['HOUSTON']
['PHILADELPHIA']
['LOS', 'ANGELES']
['MONTREAL']
SAN


In [72]:
#ranges = start+end

In [68]:
check['start_idxs'][50]

1

In [70]:
check['text'][40:55]

['W',
 'L',
 'PCT',
 'GB',
 'NEW',
 'YORK',
 '72',
 '53',
 '.576',
 '-',
 'BALTIMORE',
 '67',
 '58',
 '.536',
 '5']

In [59]:
ranges

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 2, 0, 2, 2, 0, 1, 1, 2, 0, 2, 2, 0, 1, 1, 2, 0, 2, 1, 0, 0,
       1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1, 2, 0,
       1, 0])

In [40]:
mention_ranges = []
start=False

for i in range(len(ranges)):
    if ranges[i]==1:
        if start==False:
            mention_ranges.append([i])
            start=True
        else:
            mention_ranges[-1].append(i)
            start=False
    else:
        continue
        
    

In [47]:
for x in mention_ranges:
    try:
        print(check['text'][x[0]:x[1]])
    except:
        print(check['text'][x[0]])

['Israel']
['Syria']
['JERUSALEM']
['Israel']
['Syria']
['Itamar', 'Rabinovich']
['Israel']
['Washington']
['Syria']
['Israel', 'Radio']
['Damascus']
['Syrian']
['Syrians']
['Washington']
['Rabinovich']
['Eliahu', 'Ben-Elissar']
['Israeli']
['Egypt']
['Likud']
['Israel']
['Syria']
['Washington']
['Damascus']
['Syria']
['Israel']
['Israeli']
['Damascus']
['Syrian']
['Assad']
['Israel']
['Israeli']
['David', 'Levy']
['Israel', 'Radio']
['Israeli']
['Benjamin', 'Netanyahu']
['Golan', 'Heights']
['captured', 'from']
['in', 'the', '1967']
['war', '.', 'Israeli-Syrian', 'peace', 'talks', 'have', 'been', 'deadlocked', 'over', 'the']
['since', '1991', 'despite', 'the', 'previous', 'government', "'s", 'willingness', 'to', 'make']
['concessions', '.', 'Peace', 'talks', 'between', 'the', 'two', 'sides', 'were', 'last', 'held', 'in', 'February', '.', '"', 'The', 'voices', 'coming', 'out', 'of']
['are', 'bad', ',', 'not', 'good', '.', 'The', 'media', '...', 'are', 'full', 'of', 'expressions', 'and'

In [41]:
mention_ranges

[[0, 1],
 [7, 8],
 [11, 12],
 [13, 14],
 [19, 20],
 [38, 40],
 [43, 44],
 [47, 48],
 [52, 53],
 [55, 57],
 [60, 61],
 [74, 75],
 [82, 83],
 [97, 98],
 [113, 114],
 [127, 129],
 [132, 133],
 [135, 136],
 [138, 139],
 [142, 143],
 [146, 147],
 [151, 152],
 [170, 171],
 [180, 181],
 [182, 183],
 [193, 194],
 [197, 198],
 [223, 224],
 [228, 229],
 [231, 232],
 [245, 246],
 [248, 250],
 [251, 253],
 [261, 262],
 [264, 266],
 [274, 276],
 [277, 279],
 [280, 283],
 [285, 295],
 [296, 306],
 [307, 327],
 [328, 372],
 [373, 385],
 [386, 410],
 [411, 420],
 [421, 422],
 [424, 426],
 [427, 435],
 [436, 445],
 [446, 462],
 [463, 468],
 [469, 473],
 [475, 476],
 [477]]

In [29]:
check['start_idxs'][:21]

[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]

In [30]:
check['end_idxs'][:21]

[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]

In [32]:
check['start_idxs'][21:42]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [33]:
check['end_idxs'][21:42]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [27]:
check['mentions']

['Israel',
 'Syria',
 'JERUSALEM',
 'Israel',
 'Syria',
 'Itamar Rabinovich',
 'Israel',
 'Washington',
 'Syria',
 'Israel Radio',
 'Damascus',
 'Syrian',
 'Syrians',
 'Washington',
 'Rabinovich',
 'Eliahu Ben-Elissar',
 'Israeli',
 'Egypt',
 'Likud',
 'Israel',
 'Syria',
 'Washington',
 'Damascus',
 'Syria',
 'Israel',
 'Israeli',
 'Damascus',
 'Syrian',
 'Assad',
 'Israel',
 'Israeli',
 'David Levy',
 'Israel Radio',
 'Israeli',
 'Benjamin Netanyahu',
 'Golan Heights',
 'Israel',
 'Syria',
 'Middle East',
 'Golan',
 'Golan',
 'Damascus',
 'Syria',
 'Israel',
 'God',
 'Israel',
 'Channel Two',
 'Damascus',
 'Israel',
 'Netanyahu',
 'Netanyahu',
 'Syria',
 'United States',
 'Moscow']

In [35]:
# # note: question here means mention in documents that have linked wikipedia page
# def extract_questions(filename):

#     # all the datapoints
#     global_questions = []

#     # left context so far in the document
#     left_context = []

#     # working datapoints for the document
#     document_questions = []

#     # is the entity open
#     open_entity = False

#     # question id in the document
#     question_i = 0

#     with open(filename) as fin:
#         lines = fin.readlines()

#         for line in tqdm(lines):

#             if "-DOCSTART-" in line:
#                 # new document is starting
#                 doc_id = line.split("(")[-1][:-2]

#                 # END DOCUMENT
#                 # check end of entity
#                 if open_entity:
#                     document_questions[-1]["input"].append(END_ENT_TOKEN)
#                     open_entity = False

#                 """
#                 #DEBUG
#                 for q in document_questions:
#                     pp.pprint(q)
#                     input("...")
#                 """

#                 # add sentence_questions to global_questions
#                 global_questions.extend(document_questions)

#                 # reset
#                 left_context = []
#                 document_questions = []
#                 question_i = 0

#             else:
#                 split = line.split("\t")
#                 token = split[0].strip()

#                 if len(split) >= 5:
#                     B_I = split[1]
#                     mention = split[2]
#                     #  YAGO2_entity = split[3]
#                     Wikipedia_URL = split[4]
#                     Wikipedia_ID = split[5].split('\n')[0]
#                     # Freee_base_id = split[6]

#                     if B_I == "I":
#                         pass
#                     elif B_I == "B":

#                         title = Wikipedia_URL.split("/")[-1].replace("_", " ")

#                         if Wikipedia_ID == "000":

#                             if Wikipedia_URL in url2id_cache:
#                                 pageid = url2id_cache[Wikipedia_URL]
#                             else:

#                                 pageid = _get_pageid_from_api(title)
#                                 url2id_cache[Wikipedia_URL] = pageid
#                             Wikipedia_ID = pageid

#                         q = {
#                             "id": "{}:{}".format(doc_id, question_i),
#                             "input": left_context.copy() + [BEGIN_ENT_TOKEN],
#                             "mention": mention,
#                             "Wikipedia_title": title,
#                             "Wikipedia_URL": Wikipedia_URL,
#                             "Wikipedia_ID": Wikipedia_ID,
#                             "left_context": left_context.copy(),
#                             "right_context": [],
#                         }
#                         document_questions.append(q)
#                         open_entity = True
#                         question_i += 1

#                     else:
#                         print("Invalid B_I {}", format(B_I))
#                         sys.exit(-1)
#                 else:
#                     if open_entity:
#                         document_questions[-1]["input"].append(END_ENT_TOKEN)
#                         open_entity = False

#                 left_context.append(token)
#                 for q in document_questions:
#                     q["input"].append(token)

#                 for q in document_questions[:-1]:
#                     q["right_context"].append(token)

#                 if len(document_questions) > 0 and not open_entity:
#                     document_questions[-1]["right_context"].append(token)

#     # FINAL SENTENCE
#     if open_entity:
#         document_questions[-1]["input"].append(END_ENT_TOKEN)
#         open_entity = False

#     # add sentence_questions to global_questions
#     global_questions.extend(document_questions)

#     return global_questions

In [6]:
# def convert_to_BLINK_format(questions):
#     data = []
#     for q in questions:
#         datapoint = {
#             "context_left": " ".join(q["left_context"]).strip(),
#             "mention": q["mention"],
#             "context_right": " ".join(q["right_context"]).strip(),
#             "query_id": q["id"],
#             "label_id": q["Wikipedia_ID"],
#             "Wikipedia_ID": q["Wikipedia_ID"],
#             "Wikipedia_URL": q["Wikipedia_URL"],
#             "Wikipedia_title": q["Wikipedia_title"],
#         }
#         data.append(datapoint)
#     return data

In [7]:
# # store on file
# def store_questions(questions, OUT_FILENAME):
# #     if not os.path.exists(os.path.dirname(OUT_FILENAME)):
# #         try:
# #             os.makedirs(os.path.dirname(OUT_FILENAME))
# #         except OSError as exc:  # Guard against race condition
# #             if exc.errno != errno.EEXIST:
# #                 raise
#     with open(OUT_FILENAME, "w+") as fout:
#         for q in questions:
#             json.dump(q, fout)
#             fout.write("\n")

In [8]:
# print("AIDA-YAGO2")
# in_aida_fname = 'AIDA-YAGO2-dataset.tsv'
# aida_questions = extract_questions(in_aida_fname)

  3%|▎         | 10191/323395 [00:00<00:03, 101852.06it/s]

AIDA-YAGO2


100%|██████████| 323395/323395 [00:02<00:00, 139971.91it/s]


In [9]:
train = []
testa = []
testb = []
for element in aida_questions:
    if "testa" in element["id"]:
        testa.append(element)
    elif "testb" in element["id"]:
        testb.append(element)
    else:
        train.append(element)
print("train: {}".format(len(train)))
print("testa: {}".format(len(testa)))
print("testb: {}".format(len(testb)))

train: 18541
testa: 4791
testb: 4485


In [10]:
train_blink = convert_to_BLINK_format(train)
testa_blink = convert_to_BLINK_format(testa)
testb_blink = convert_to_BLINK_format(testb)

In [11]:
out_train_aida_filename = "AIDA-YAGO2_train.jsonl"
store_questions(train_blink, out_train_aida_filename)
out_testa_aida_filename = "AIDA-YAGO2_testa.jsonl"
store_questions(testa_blink, out_testa_aida_filename)
out_testb_aida_filename = "AIDA-YAGO2_testb.jsonl"
store_questions(testb_blink, out_testb_aida_filename)